In [1]:
import $ivy.`org.scalatest::scalatest:3.0.5`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

# 1. Using implicit arguments

Define a case class Car, with one member `_speed` that handles the speed internally in km/h. This class will offer support to work with other measure units.

These are the measures we are going to give support to

## PART I

Implement a private method `toKPH` that gets a quantity and an implicit measure, does the conversion to km/h.

| value | conversion |
| --- | --- |
| 1 km/h | 1        km/h |
| 1 m/s  | 3.6      km/h |
| 1 mph  | 1.609344 km/h |
| 1 knot | 1.852    km/h |

## PART II

Implement the opposite, a private method `fromKPH` that gets a quantity and an implicit measure, and does the conversion from km/h to the implicit measure.

## PART III

Implement a method `speedUp` that given an increment and an implicit measure, increments the car's speed accordingly.

## PART IV

Implement a method `speed` that returns the current speed (_speed) converting to the implicit measure received as argument

In [4]:
sealed abstract class MeasureUnit
case object KilometersPerHour extends MeasureUnit
case object MetersPerSecond extends MeasureUnit
case object MilesPerHour extends MeasureUnit
case object Knots extends MeasureUnit

case class Car(_speed: Double) {
    private def toKPH(q: Double)(implicit U: MeasureUnit): Double = U match {
        case KilometersPerHour => q
        case MetersPerSecond => q * 3.6
        case MilesPerHour => q * 1.609344
        case Knots => q * 1.852
        }

    private def fromKPH(q: Double)(implicit U: MeasureUnit): Double = U match {
        case KilometersPerHour => q
        case MetersPerSecond => q / 3.6
        case MilesPerHour => q / 1.609344
        case Knots => q / 1.852
        }

    def speedUp(inc: Double)(implicit U: MeasureUnit): Car = Car(_speed+toKPH(inc))
    def speed(implicit U: MeasureUnit): Double = fromKPH(_speed)
  }

defined class MeasureUnit
defined object KilometersPerHour
defined object MetersPerSecond
defined object MilesPerHour
defined object Knots
defined class Car

In [5]:
case class Test1() extends FunSpec with Matchers {
    val car100 = Car(100)
  describe("Testing speedUp & speed") {
    describe("Using Km/h as default") {
      implicit val defaultMeasurementUnits = KilometersPerHour

      it("should speed up w/o changing measures") {
        car100.speedUp(10)._speed shouldBe 110
        car100.speedUp(10).speed shouldBe 110
      }

      it("should increment 3.6 times when using m/s explicitly") {
        car100.speedUp(10)(MetersPerSecond).speed shouldBe 136
      }
    }

    describe("Using m/s as default") {
      implicit val defaultMeasurementUnits = MetersPerSecond

      it("should speed up changing measures") {
        car100.speedUp(10)._speed shouldBe 136
        car100.speedUp(10).speed shouldBe (136 / 3.6)
      }

      it("should decrement 3.6 times when using km/h explicitly") {
        car100.speedUp(10)(KilometersPerHour)._speed shouldBe 110
        car100.speedUp(10)(KilometersPerHour).speed shouldBe (110 / 3.6)
      }
    }
  }
}

run(Test1())

cmd4$Helper$Test1:
Testing speedUp & speed
  Using Km/h as default
  - should speed up w/o changing measures
  - should increment 3.6 times when using m/s explicitly
  Using m/s as default
  - should speed up changing measures
  - should decrement 3.6 times when using km/h explicitly


defined class Test1

# 2. Using implicit classes

## PART V

Imagine we don't have access to the Car class and we need to extend its functionality with a method `stop` that will set the speed to 0. 

In [6]:
implicit class CarStopper(car: Car) {
    def stop:Car = car.speedUp(-car.speed(KilometersPerHour))(KilometersPerHour)
}

case class Test2() extends FunSpec with Matchers {
    val car100 = Car(100)
    describe("The stop method") {
        it("should set the speed to 0") {
          car100.stop._speed shouldBe 0
        }
  }
}

run(Test2())

cmd5$Helper$Test2:
The stop method
- should set the speed to 0


defined class CarStopper
defined class Test2